In [1]:
using JuMP, Gurobi, DataFrames, CSV, MLDataUtils

INFO: Recompiling stale cache file /Users/wenweiliu/.julia/lib/v0.6/MLDataUtils.ji for module MLDataUtils.


In [2]:
data = CSV.read("Ready_insurance.csv",header = true);

In [3]:
X = Matrix(data[1:end - 1])
y = data[end];
# Split into training, test sets (50%/25%/25%)
n = length(y)
train_val = round(Int, 0.5 * n)
val_test = round(Int, 0.75 * n)
# Training set X, y
X_train = X[1:train_val - 1, :]
y_train = y[1:train_val - 1]
# Cross Validation set X, y
X_val = X[train_val:val_test - 1, :]
y_val = y[train_val:val_test - 1]
# Test set X, y
X_test = X[val_test:end, :]
y_test = y[val_test:end];

In [4]:
function p2_reg_2(X, y, ρ)
    m = Model(solver=GurobiSolver(OutputFlag=0))
    
    p = size(X, 2)

    # Variables
    @variable(m, t_1)
    @variable(m, t_2)
    @variable(m, β[1:p])
    
    # Constraints
    @constraint(m, norm(y - X * β) <= t_1)
    @constraint(m, ρ * norm(β) <= t_2)

    # Objective
    @objective(m, Min, t_1+t_2)

    solve(m)
    
    return getvalue(β)
end

p2_reg_2 (generic function with 1 method)

In [5]:
function det_outlier(X,K)
    n, p = size(X)
    m = Model(solver = GurobiSolver(OutputFlag=0))
    @variable(m, z[1:n], Bin);
    @constraint(m, sum(z) == (n-K));
    @objective(m,Min,sum(((X.-median(X,1)).*z).^2))
    solve(m)
    return(getobjectivevalue(m),getvalue(z))
end

det_outlier (generic function with 1 method)

In [6]:
ρ_list = [0.001, 0.01, 0.1, 1, 2]
for i in 1:5
    β_soln = p2_reg_2(X_train, y_train, ρ_list[i])
    y_hat_train = X_train*β_soln
    R2_train = 1 - sum((y_hat_train - y_train).^2) / sum((mean(y_train) - y_train).^2)
    y_hat_val = X_val*β_soln
    R2_val = 1 - sum((y_hat_val - y_val).^2) / sum((mean(y_train) - y_train).^2)
    println("With pentalty: ",ρ_list[i])
    println("R-squared for traning: ", R2_train)
    println("R-squared for cross-validation: ", R2_val)
end

Academic license - for non-commercial use only
With pentalty: 0.001
R-squared for traning: 0.30273981088120006
R-squared for cross-validation: 0.6233004966378886
Academic license - for non-commercial use only
With pentalty: 0.01
R-squared for traning: 0.30273920543070565
R-squared for cross-validation: 0.6233028267061882
Academic license - for non-commercial use only
With pentalty: 0.1
R-squared for traning: 0.3026702314547206
R-squared for cross-validation: 0.6233909150781914
Academic license - for non-commercial use only
With pentalty: 1.0
R-squared for traning: 0.2959056785552684
R-squared for cross-validation: 0.6211829201913291
Academic license - for non-commercial use only
With pentalty: 2.0
R-squared for traning: 0.2755756134943198
R-squared for cross-validation: 0.6125429201116139


In [8]:
n, p = size(X_train)
for i in 1:
# obj_out, z_list = det_outlier(X_train,)
# X_train2 = X_train[]

(668, 11)